In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [2]:
# 1.tf.train.Feature(bytes_list, float_list, int64_list)
book = [name.encode("utf-8") for name in ["machine learning", "C++"]]
book_list = tf.train.BytesList(value = book)
print(book_list)

hours = [1.0, 2.0, 3.0, 4.0]
hours_floatlist = tf.train.FloatList(value = hours)
print(hours_floatlist)

count = [9]
count_Intlist = tf.train.Int64List(value = count)
print(count_Intlist)

value: "machine learning"
value: "C++"

value: 1.0
value: 2.0
value: 3.0
value: 4.0

value: 9



In [3]:
# 2.tf.train.Features(feature = {"key": XXXX})
features = tf.train.Features(feature={
    "book_list": tf.train.Feature(bytes_list = book_list),
    "hours_floatlist": tf.train.Feature(float_list = hours_floatlist),
    "count_Intlist": tf.train.Feature(int64_list = count_Intlist)
})

print(features)

feature {
  key: "book_list"
  value {
    bytes_list {
      value: "machine learning"
      value: "C++"
    }
  }
}
feature {
  key: "count_Intlist"
  value {
    int64_list {
      value: 9
    }
  }
}
feature {
  key: "hours_floatlist"
  value {
    float_list {
      value: 1.0
      value: 2.0
      value: 3.0
      value: 4.0
    }
  }
}



In [4]:
# 3.tf.train.Example
example = tf.train.Example(features = features)

In [5]:
# 4.example的序列化
serialize_example = example.SerializeToString()
print(serialize_example)

b"\ni\n&\n\tbook_list\x12\x19\n\x17\n\x10machine learning\n\x03C++\n'\n\x0fhours_floatlist\x12\x14\x12\x12\n\x10\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\n\x16\n\rcount_Intlist\x12\x05\x1a\x03\n\x01\t"


In [6]:
# 生成文件名
output_dir = "tfrecord"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
file_name = "test.tfrecords"
fullfile_name = os.path.join(output_dir, file_name)

# 写入文件 TFRecordWriter
with tf.io.TFRecordWriter(path = fullfile_name) as writer:
    for _ in range(3):
        writer.write(serialize_example)

In [7]:
# 生成dataset TFRecordDataset
dataset = tf.data.TFRecordDataset(filenames=[fullfile_name])

# 读取序列化文件
for line in dataset:
    print(line)

tf.Tensor(b"\ni\n&\n\tbook_list\x12\x19\n\x17\n\x10machine learning\n\x03C++\n'\n\x0fhours_floatlist\x12\x14\x12\x12\n\x10\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\n\x16\n\rcount_Intlist\x12\x05\x1a\x03\n\x01\t", shape=(), dtype=string)
tf.Tensor(b"\ni\n&\n\tbook_list\x12\x19\n\x17\n\x10machine learning\n\x03C++\n'\n\x0fhours_floatlist\x12\x14\x12\x12\n\x10\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\n\x16\n\rcount_Intlist\x12\x05\x1a\x03\n\x01\t", shape=(), dtype=string)
tf.Tensor(b"\ni\n&\n\tbook_list\x12\x19\n\x17\n\x10machine learning\n\x03C++\n'\n\x0fhours_floatlist\x12\x14\x12\x12\n\x10\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\n\x16\n\rcount_Intlist\x12\x05\x1a\x03\n\x01\t", shape=(), dtype=string)


In [8]:
#读取原来文件内容
except_feature = {
    "book_list": tf.io.VarLenFeature(dtype=tf.string),
    "hours_floatlist": tf.io.VarLenFeature(dtype=tf.float32),
    "count_Intlist": tf.io.FixedLenFeature([], dtype=tf.int64),
}

for serialized_example_tensor in dataset:
    example = tf.io.parse_single_example(serialized=serialized_example_tensor, features=except_feature)
    books = tf.sparse.to_dense(example["book_list"])
    for book in books:
        print(book.numpy().decode("utf-8"))

machine learning
C++
machine learning
C++
machine learning
C++


In [9]:
# 写入压缩文件
output_dir = "tfrecord"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
file_name = "test.tfrecords" + ".zip"
fullfile_name_zip = os.path.join(output_dir, file_name)

# 定义压缩文件类型
options = tf.io.TFRecordOptions(compression_type="GZIP")

# 写入压缩文件 TFRecordWriter
with tf.io.TFRecordWriter(path = fullfile_name_zip, options=options) as writer:
    for _ in range(3):
        writer.write(serialize_example)

In [10]:
# 读取压缩文件 compression_type
dataset_zip = tf.data.TFRecordDataset(filenames=[fullfile_name_zip],compression_type="GZIP")
for serialized_example_tensor in dataset_zip:
    example = tf.io.parse_single_example(serialized=serialized_example_tensor, features=except_feature)
    books = tf.sparse.to_dense(example["book_list"])
    for book in books:
        print(book.numpy().decode("utf-8"))

machine learning
C++
machine learning
C++
machine learning
C++
